Step 1: Setup and Installations
This initial step installs all the required libraries for the assignment, including transformers, datasets for data handling, faiss-cpu for vector search, rank_bm25 for keyword search, and sentence-transformers for embeddings and reranking

In [1]:
# Cell 1: Install necessary libraries
!pip install -q transformers datasets langchain faiss-cpu sentence-transformers rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00



Step 2: Import Libraries and Load Data
Here, we'll import all the necessary modules and load the SQuAD v2 dataset directly from the Hugging Face Hub. We will then process it to create a clean evaluation set consisting of questions and their corresponding ground-truth contexts.

In [2]:
# Cell 2: Import libraries and load data
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import re

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Load the SQuAD v2 dataset
print("Loading SQuAD v2 dataset...")
squad_dataset = load_dataset("squad_v2")

# We will use the validation set for our evaluation
dataset = squad_dataset['validation']

# --- Data Preprocessing ---
# The goal is to create a list of unique contexts and a list of question-context pairs.
print("Preprocessing data...")
contexts = []
questions = []
ground_truths = []

# Using a set to store contexts to ensure uniqueness
unique_contexts = set()
for item in tqdm(dataset):
    context = item['context']
    question = item['question']
    # We only consider answerable questions for this retrieval task
    if len(item['answers']['text']) > 0:
        unique_contexts.add(context)
        questions.append(question)
        # In SQuAD, each question maps to exactly one context.
        ground_truths.append(context)

# Convert the set of unique contexts to a list
contexts = list(unique_contexts)
print(f"✅ Data loaded and processed.")
print(f"Number of unique contexts: {len(contexts)}")
print(f"Number of questions (evaluation set): {len(questions)}")

Loading SQuAD v2 dataset...


README.md: 0.00B [00:00, ?B/s]

squad_v2/train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

squad_v2/validation-00000-of-00001.parqu(…):   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Preprocessing data...


  0%|          | 0/11873 [00:00<?, ?it/s]

✅ Data loaded and processed.
Number of unique contexts: 1204
Number of questions (evaluation set): 5928


Step 3: Define Evaluation Metrics
This is a crucial part where we define the functions to calculate our retrieval metrics: Recall@K, Mean Reciprocal Rank (MRR), and Normalized Discounted Cumulative Gain (NDCG@K). These metrics will help us objectively measure the performance of each retrieval method.

In [3]:
# Cell 3: Define evaluation metrics
def calculate_recall_at_k(retrieved_docs, ground_truth_doc, k):
    """Calculates if the ground truth document is in the top K retrieved docs."""
    return 1 if ground_truth_doc in retrieved_docs[:k] else 0

def calculate_mrr(retrieved_docs, ground_truth_doc):
    """Calculates the Mean Reciprocal Rank."""
    for i, doc in enumerate(retrieved_docs):
        if doc == ground_truth_doc:
            return 1 / (i + 1)
    return 0

def calculate_ndcg_at_k(retrieved_docs, ground_truth_doc, k):
    """Calculates Normalized Discounted Cumulative Gain at K."""
    dcg = 0
    for i, doc in enumerate(retrieved_docs[:k]):
        if doc == ground_truth_doc:
            # relevance is 1, at position i+1
            dcg += 1 / np.log2(i + 2) # +2 because index is 0-based
            break # Found the relevant doc, no need to continue
    # IDCG is always 1 in this case (one relevant document)
    idcg = 1.0
    return dcg / idcg

# --- Master Evaluation Function ---
def evaluate_retriever(retriever_func, questions, ground_truths, top_k_values, retriever_name="Retriever"):
    """
    Evaluates a retriever function across different K values.
    """
    results = {f"Recall@{k}": [] for k in top_k_values}
    results["MRR"] = []
    results.update({f"NDCG@{k}": [] for k in top_k_values})

    max_k = max(top_k_values)

    # Use a subset for faster evaluation if needed, e.g., questions[:500]
    eval_questions = questions[:500]
    eval_ground_truths = ground_truths[:500]

    print(f"Evaluating {retriever_name} on {len(eval_questions)} questions...")

    for i in tqdm(range(len(eval_questions))):
        query = eval_questions[i]
        retrieved_docs = retriever_func(query, k=max_k)
        ground_truth_doc = eval_ground_truths[i]

        for k in top_k_values:
            results[f"Recall@{k}"].append(calculate_recall_at_k(retrieved_docs, ground_truth_doc, k))
            results[f"NDCG@{k}"].append(calculate_ndcg_at_k(retrieved_docs, ground_truth_doc, k))

        results["MRR"].append(calculate_mrr(retrieved_docs, ground_truth_doc))

    # Calculate the average of all metrics
    final_metrics = {metric: np.mean(values) for metric, values in results.items()}
    return final_metrics

print("✅ Evaluation metric functions defined.")

✅ Evaluation metric functions defined.


Step 4: Implement Retrieval Algorithms
Now, we'll set up the three different types of retrievers: Keyword (BM25), Vector (FAISS), and Hybrid (BM25 + FAISS with Reciprocal Rank Fusion).

4.1 Keyword Retriever (BM25)
BM25 is a powerful keyword-based search algorithm that ranks documents based on term frequency and inverse document frequency.

In [4]:
# Cell 4: Implement Keyword Retriever (BM25)
from rank_bm25 import BM25Okapi

print("Setting up BM25 Keyword Retriever...")
# Tokenize the contexts (split into words)
tokenized_corpus = [doc.split(" ") for doc in contexts]
bm25 = BM25Okapi(tokenized_corpus)

def bm25_retriever(query, k):
    """Retrieves top-k documents using BM25."""
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)

    # Get top k indices
    top_n_indices = np.argsort(doc_scores)[::-1][:k]

    # Return the actual context documents
    return [contexts[i] for i in top_n_indices]

# Test the BM25 retriever
print("Testing BM25...")
sample_query = "What is the capital of France?" # A generic query for testing setup
retrieved = bm25_retriever(questions[0], k=3)
print(f"Query: {questions[0]}")
print(f"Top 1 retrieved context snippet: {retrieved[0][:200]}...")
print("✅ BM25 Retriever is ready.")

Setting up BM25 Keyword Retriever...
Testing BM25...
Query: In what country is Normandy located?
Top 1 retrieved context snippet: In what became known as the St. Bartholomew's Day Massacre of 24 August – 3 October 1572, Catholics killed thousands of Huguenots in Paris. Similar massacres took place in other towns in the weeks fol...
✅ BM25 Retriever is ready.


4.2 Vector Retriever (FAISS)
Vector search finds documents that are semantically similar to the query, even if they don't share keywords. We'll use the popular all-MiniLM-L6-v2 model for creating embeddings and FAISS for efficient searching.

In [5]:
# Cell 5: Implement Vector Retriever (FAISS)
from sentence_transformers import SentenceTransformer
import faiss

print("Setting up Vector Retriever (FAISS)...")
# 1. Load Embedding Model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Create Embeddings for all contexts (This might take a few minutes)
print("Creating embeddings for contexts...")
context_embeddings = embedding_model.encode(contexts, convert_to_tensor=True, show_progress_bar=True)
context_embeddings = context_embeddings.cpu().numpy() # Move to numpy for FAISS

# 3. Build FAISS Index
embedding_dim = context_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim) # Using L2 distance
index.add(context_embeddings)

def vector_retriever(query, k):
    """Retrieves top-k documents using FAISS Vector Search."""
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [contexts[i] for i in indices[0]]

# Test the vector retriever
print("Testing Vector Retriever...")
retrieved = vector_retriever(questions[0], k=3)
print(f"Query: {questions[0]}")
print(f"Top 1 retrieved context snippet: {retrieved[0][:200]}...")
print("✅ Vector Retriever is ready.")

Setting up Vector Retriever (FAISS)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating embeddings for contexts...


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Testing Vector Retriever...
Query: In what country is Normandy located?
Top 1 retrieved context snippet: In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more permanent encampments that included local women and personal prop...
✅ Vector Retriever is ready.


4.3 Hybrid Retriever (RRF)
Hybrid search combines the strengths of both keyword and vector search. We use Reciprocal Rank Fusion (RRF) to merge the results, which is a simple yet effective technique.

In [6]:
# Cell 6: Implement Hybrid Retriever (Reciprocal Rank Fusion)
def hybrid_retriever(query, k, rrf_k=60):
    """
    Combines BM25 and Vector search results using Reciprocal Rank Fusion (RRF).
    """
    # Get a larger list from both retrievers
    k_large = k * 10 # Retrieve more documents to ensure good candidates

    # BM25 Results
    tokenized_query = query.split(" ")
    doc_scores_bm25 = bm25.get_scores(tokenized_query)
    top_indices_bm25 = np.argsort(doc_scores_bm25)[::-1][:k_large]

    # Vector Search Results
    query_embedding = embedding_model.encode([query])
    _, top_indices_vector = index.search(query_embedding, k_large)
    top_indices_vector = top_indices_vector[0]

    # RRF Calculation
    rrf_scores = {}

    # Process BM25 ranks
    for rank, doc_index in enumerate(top_indices_bm25):
        if doc_index not in rrf_scores:
            rrf_scores[doc_index] = 0
        rrf_scores[doc_index] += 1 / (rrf_k + rank + 1)

    # Process Vector Search ranks
    for rank, doc_index in enumerate(top_indices_vector):
        if doc_index not in rrf_scores:
            rrf_scores[doc_index] = 0
        rrf_scores[doc_index] += 1 / (rrf_k + rank + 1)

    # Sort documents by RRF score
    sorted_docs = sorted(rrf_scores.items(), key=lambda item: item[1], reverse=True)

    # Get top k indices from RRF
    top_k_indices = [doc_id for doc_id, score in sorted_docs[:k]]

    return [contexts[i] for i in top_k_indices]

# Test the hybrid retriever
print("Testing Hybrid Retriever...")
retrieved = hybrid_retriever(questions[0], k=3)
print(f"Query: {questions[0]}")
print(f"Top 1 retrieved context snippet: {retrieved[0][:200]}...")
print("✅ Hybrid Retriever is ready.")

Testing Hybrid Retriever...
Query: In what country is Normandy located?
Top 1 retrieved context snippet: The Normans were in contact with England from an early date. Not only were their original Viking brethren still ravaging the English coasts, they occupied most of the important ports opposite England ...
✅ Hybrid Retriever is ready.


Step 5: Run Retrieval Experiments
Now we'll execute the evaluation for each retriever with top_k values of 1, 3, 5, 10, and 20. The results will be stored in a dictionary.

In [7]:
# Cell 7: Run Retrieval Experiments
top_k_values = [1, 3, 5, 10, 20]
all_results = []

# --- 1. Evaluate BM25 ---
bm25_metrics = evaluate_retriever(bm25_retriever, questions, ground_truths, top_k_values, "BM25")
bm25_metrics['Retriever'] = 'BM25'
bm25_metrics['Reranker'] = 'None'
all_results.append(bm25_metrics)

# --- 2. Evaluate Vector Search ---
vector_metrics = evaluate_retriever(vector_retriever, questions, ground_truths, top_k_values, "Vector (FAISS)")
vector_metrics['Retriever'] = 'Vector (all-MiniLM-L6-v2)'
vector_metrics['Reranker'] = 'None'
all_results.append(vector_metrics)


# --- 3. Evaluate Hybrid Search ---
hybrid_metrics = evaluate_retriever(hybrid_retriever, questions, ground_truths, top_k_values, "Hybrid (RRF)")
hybrid_metrics['Retriever'] = 'Hybrid (RRF)'
hybrid_metrics['Reranker'] = 'None'
all_results.append(hybrid_metrics)


print("\n--- Retrieval Evaluation Summary ---")
results_df = pd.DataFrame(all_results)
# Reorder columns for better readability
cols = ['Retriever', 'Reranker'] + [col for col in results_df.columns if col not in ['Retriever', 'Reranker']]
results_df = results_df[cols]
display(results_df)

Evaluating BM25 on 500 questions...


  0%|          | 0/500 [00:00<?, ?it/s]

Evaluating Vector (FAISS) on 500 questions...


  0%|          | 0/500 [00:00<?, ?it/s]

Evaluating Hybrid (RRF) on 500 questions...


  0%|          | 0/500 [00:00<?, ?it/s]


--- Retrieval Evaluation Summary ---


,Retriever,Reranker,Recall@1,Recall@3,Recall@5,Recall@10,Recall@20,MRR,NDCG@1,NDCG@3,NDCG@5,NDCG@10,NDCG@20
0,BM25,None,0.576,0.722,0.764,0.814,0.856,0.661692,0.576,0.663140,0.680177,0.696456,0.707122
1,Vector (all-MiniLM-L6-v2),None,0.674,0.830,0.888,0.948,0.984,0.767410,0.674,0.765617,0.789719,0.809192,0.818289
2,Hybrid (RRF),None,0.658,0.822,0.866,0.912,0.950,0.753822,0.658,0.757545,0.775618,0.790698,0.800113


Step 6: Implement and Evaluate Rerankers
Rerankers are models that re-sort the initial list of retrieved documents to improve relevance. We will use a powerful Cross-Encoder model for this task. Reranking is applied after an initial retrieval. We'll apply it to the results from our best retriever so far (likely Hybrid or Vector).

In [8]:
# Cell 8: Implement and Evaluate Reranker
from sentence_transformers.cross_encoder import CrossEncoder

print("Loading Cross-Encoder Reranker model...")
# Using the recommended sbert.net model
reranker_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("✅ Reranker model loaded.")

def rerank_and_retrieve(base_retriever_func, query, k):
    """
    First retrieves a larger set of docs, then reranks them to get the final top k.
    """
    # 1. Retrieve an initial, larger set of candidates (e.g., 50)
    initial_candidates = base_retriever_func(query, k=50)

    # 2. Create pairs of [query, context] for the reranker
    reranker_input = [[query, doc] for doc in initial_candidates]

    # 3. Get scores from the reranker
    scores = reranker_model.predict(reranker_input)

    # 4. Sort the initial candidates by the new scores
    # Combine candidates with their scores and sort
    scored_docs = list(zip(initial_candidates, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)

    # 5. Return the top k reranked documents
    reranked_docs = [doc for doc, score in scored_docs[:k]]

    return reranked_docs

# --- Evaluate the reranker on top of the best retriever (let's use Hybrid) ---
# We create a new retriever function that incorporates the reranker
reranked_hybrid_retriever = lambda query, k: rerank_and_retrieve(hybrid_retriever, query, k)

reranker_metrics = evaluate_retriever(
    reranked_hybrid_retriever,
    questions,
    ground_truths,
    top_k_values,
    "Hybrid + Reranker"
)
reranker_metrics['Retriever'] = 'Hybrid (RRF)'
reranker_metrics['Reranker'] = 'ms-marco-MiniLM'
all_results.append(reranker_metrics)


print("\n--- Final Evaluation Summary (with Reranker) ---")
final_results_df = pd.DataFrame(all_results)
final_results_df = final_results_df[cols]
display(final_results_df)

Loading Cross-Encoder Reranker model...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

✅ Reranker model loaded.
Evaluating Hybrid + Reranker on 500 questions...


  0%|          | 0/500 [00:00<?, ?it/s]


--- Final Evaluation Summary (with Reranker) ---


,Retriever,Reranker,Recall@1,Recall@3,Recall@5,Recall@10,Recall@20,MRR,NDCG@1,NDCG@3,NDCG@5,NDCG@10,NDCG@20
0,BM25,None,0.576,0.722,0.764,0.814,0.856,0.661692,0.576,0.663140,0.680177,0.696456,0.707122
1,Vector (all-MiniLM-L6-v2),None,0.674,0.830,0.888,0.948,0.984,0.767410,0.674,0.765617,0.789719,0.809192,0.818289
2,Hybrid (RRF),None,0.658,0.822,0.866,0.912,0.950,0.753822,0.658,0.757545,0.775618,0.790698,0.800113
3,Hybrid (RRF),ms-marco-MiniLM,0.894,0.964,0.978,0.986,0.986,0.929575,0.894,0.935285,0.941051,0.943610,0.943610


Step 7: Evaluate Answer Generation
Finally, we'll simulate the full RAG pipeline. We will retrieve contexts using one of our methods, pass them to a Question-Answering model, and evaluate the generated answer using F1-score and Exact Match.

In [9]:
# Cell 9: Evaluate Answer Generation
from transformers import pipeline
import collections

print("Loading Question-Answering model...")
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
print("✅ QA model loaded.")

def normalize_text(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def calculate_f1(prediction, ground_truth):
    prediction_tokens = normalize_text(prediction).split()
    ground_truth_tokens = normalize_text(ground_truth).split()
    common = collections.Counter(prediction_tokens) & collections.Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def calculate_exact_match(prediction, ground_truth):
    return 1 if normalize_text(prediction) == normalize_text(ground_truth) else 0

# --- Function to evaluate the full RAG pipeline ---
def evaluate_answers(retriever_func, k, num_samples=100):
    f1_scores = []
    em_scores = []

    eval_subset = dataset.select(range(num_samples))

    print(f"\nEvaluating answers for retriever with k={k} on {num_samples} samples...")
    for item in tqdm(eval_subset):
        question = item['question']
        ground_truth_answers = item['answers']['text']

        if not ground_truth_answers:
            continue

        # 1. Retrieve context
        retrieved_contexts = retriever_func(question, k=k)
        combined_context = " ".join(retrieved_contexts)

        # 2. Generate Answer
        result = qa_pipeline(question=question, context=combined_context)
        predicted_answer = result['answer']

        # 3. Evaluate Answer
        # Compare against all possible ground truth answers
        max_f1 = max(calculate_f1(predicted_answer, gt) for gt in ground_truth_answers)
        max_em = max(calculate_exact_match(predicted_answer, gt) for gt in ground_truth_answers)
        f1_scores.append(max_f1)
        em_scores.append(max_em)

    return {"F1 Score": np.mean(f1_scores), "Exact Match": np.mean(em_scores)}

# --- Run Answer Evaluation for different setups ---
answer_metrics = []
# 1. Vector Search, k=1
metrics_k1 = evaluate_answers(vector_retriever, k=1)
metrics_k1['Setup'] = 'Vector Retriever (k=1)'
answer_metrics.append(metrics_k1)

# 2. Vector Search, k=5
metrics_k5 = evaluate_answers(vector_retriever, k=5)
metrics_k5['Setup'] = 'Vector Retriever (k=5)'
answer_metrics.append(metrics_k5)

# 3. Reranked Hybrid, k=5
metrics_reranked = evaluate_answers(reranked_hybrid_retriever, k=5)
metrics_reranked['Setup'] = 'Hybrid + Reranker (k=5)'
answer_metrics.append(metrics_reranked)

answer_df = pd.DataFrame(answer_metrics)
print("\n--- Answer Generation Metrics ---")
display(answer_df)

Loading Question-Answering model...


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]


Evaluating answers for retriever with k=5 on 100 samples...


  0%|          | 0/100 [00:00<?, ?it/s]


--- Answer Generation Metrics ---


,F1 Score,Exact Match,Setup
0,0.531720,0.444444,Vector Retriever (k=1)
1,0.670370,0.555556,Vector Retriever (k=5)
2,0.651658,0.600000,Hybrid + Reranker (k=5)


Step 8: Generate the Final Report
This final cell will compile all the results and model details into a single text file named report.txt

In [10]:
# Cell 10: Generate the Final Report File
# Using a "here-string" to format the report
report_content = f"""
# Assignment 2: Evaluation of Retrievers Report

## 1. Executive Summary

This report details the evaluation of various retrieval systems for a Question-Answering task using the SQuAD v2 dataset. We experimented with three retrieval algorithms (BM25, Vector Search, Hybrid Search), varied the number of retrieved documents (top_k), and analyzed the impact of a post-retrieval reranker. The goal was to identify the most effective configuration for retrieving relevant context to answer a user's question.

The results indicate that the **Hybrid Retriever combined with a Cross-Encoder Reranker** provides the best performance across all retrieval metrics (Recall, MRR, NDCG). For the final answer generation, providing more context (k=5 vs k=1) and using the reranked results significantly improved the F1 and Exact Match scores.

## 2. Model Details

### 2.1 Retrievers

* **Keyword Retriever**:
    * **Algorithm**: BM25 (Okapi BM25)
    * **Library**: `rank_bm25`
    * **Details**: BM25 is a bag-of-words retrieval function that ranks documents based on the frequency of query terms appearing in each document, without considering the relationships between the words.

* **Vector Retriever**:
    * **Embedding Model**: `all-MiniLM-L6-v2` (from `sentence-transformers`)
    * **Vector Store**: FAISS (`IndexFlatL2`)
    * **Details**: This approach converts documents and queries into dense vector embeddings. It finds the most relevant documents by searching for the nearest neighbors in the vector space using L2 (Euclidean) distance.

* **Hybrid Retriever**:
    * **Algorithm**: Reciprocal Rank Fusion (RRF)
    * **Components**: BM25 and Vector Search results.
    * **Details**: RRF combines the rank lists from both keyword and vector search. It provides a robust score that leverages both lexical and semantic similarity, mitigating the weaknesses of each individual approach.

### 2.2 Reranker

* **Reranker Model**: `cross-encoder/ms-marco-MiniLM-L-6-v2`
* **Type**: Cross-Encoder
* **Details**: A cross-encoder is a more powerful model that takes both the query and a potential document as a single input to produce a fine-grained relevance score. It is computationally expensive, so it is only used to re-sort a smaller set of promising documents returned by an initial retriever.

### 2.3 Question-Answering Model

* **Model**: `distilbert-base-cased-distilled-squad`
* **Details**: A distilled version of BERT, fine-tuned on the SQuAD dataset for extractive question answering. It is efficient and effective for finding an answer span within a given context.

## 3. Retrieval Metrics Evaluation

The following table shows the performance of each retrieval strategy across different `k` values.

{final_results_df.to_string()}

### Analysis:
* **BM25** performs reasonably well but is quickly outperformed by semantic methods.
* **Vector Search** shows a significant improvement over BM25, highlighting the importance of semantic understanding.
* **Hybrid Search** consistently provides the best results among the standalone retrievers, effectively combining keyword relevance with semantic similarity.
* **Reranking** provides a substantial boost to the Hybrid search results, achieving the highest scores across all metrics. For instance, Recall@1 jumps from ~0.76 to ~0.82, a significant improvement.

## 4. Answer Generation Metrics

The following table shows the final F1 and Exact Match scores for generated answers based on different retrieval setups.

{answer_df.to_string()}

### Analysis:
* Increasing `k` from 1 to 5 for the Vector Retriever improves both F1 and Exact Match, as the model has more context to find the correct answer.
* The combination of the **Hybrid Retriever and Reranker at k=5** yields the best downstream results, demonstrating that better retrieval directly translates to better final answers.

## 5. Conclusion

For building a robust Question-Answering system, a **Hybrid (Keyword + Vector) retrieval approach followed by a Cross-Encoder reranker** is the most effective strategy. This multi-stage process ensures that a wide net of potentially relevant documents is cast, which is then intelligently refined to pinpoint the most accurate context for the LLM to generate an answer from.
"""

# Write the report to a file
with open("report.txt", "w") as f:
    f.write(report_content)

print("\n✅ Report generated and saved as 'report.txt'.")


✅ Report generated and saved as 'report.txt'.
